# ✔ 10\. [**Pièges courants et pratiques recommandées**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/10_pitfalls_good_practises.ipynb)<br/>([*Common pitfalls and recommended practices*](https://scikit-learn.org/stable/common_pitfalls.html))

Le but de ce chapitre est d'illustrer certains pièges et anti-modèles courants qui se produisent lors de l'utilisation de scikit-learn. Il fournit des exemples de ce qu'il **ne faut pas** faire, ainsi qu'un exemple correct correspondant.


✔ 10.1. [**Prétraitement incohérent**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/10_pitfalls_good_practises.ipynb#inconsistent-preprocessing)
([*Inconsistent preprocessing*](https://scikit-learn.org/stable/common_pitfalls.html#inconsistent-preprocessing))

✔ 10.2. [**Fuite de données**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/10_pitfalls_good_practises.ipynb#data-leakage)
([*Data leakage*](https://scikit-learn.org/stable/common_pitfalls.html#data-leakage))
* ✔ 10.2.1. [**Fuite de données lors du prétraitement**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/10_pitfalls_good_practises.ipynb#data-leakage-during-pre-processing)
([*Data leakage during pre-processing*](https://scikit-learn.org/stable/common_pitfalls.html#data-leakage-during-pre-processing))
* ✔ 10.2.2. [**Comment éviter les fuites de données**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/10_pitfalls_good_practises.ipynb#how-to-avoid-data-leakage)
([*How to avoid data leakage*](https://scikit-learn.org/stable/common_pitfalls.html#how-to-avoid-data-leakage))

✔ 10.3. [**Contrôle de l'aléatoire**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/10_pitfalls_good_practises.ipynb#controlling-randomness)
([*Controlling randomness*](https://scikit-learn.org/stable/common_pitfalls.html#controlling-randomness))
* ✔ 10.3.1. [**Utilisation d'instances None ou RandomState, et appels répétés pour ajuster et diviser**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/10_pitfalls_good_practises.ipynb#using-none-or-randomstate-instances-and-repeated-calls-to-fit-and-split)
([*Using None or RandomState instances, and repeated calls to fit and split*](https://scikit-learn.org/stable/common_pitfalls.html#using-none-or-randomstate-instances-and-repeated-calls-to-fit-and-split))
* ✔ 10.3.2. [**Pièges et subtilités courants**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/10_pitfalls_good_practises.ipynb#common-pitfalls-and-subtleties)
([*Common pitfalls and subtleties*](https://scikit-learn.org/stable/common_pitfalls.html#common-pitfalls-and-subtleties))
* ✔ 10.3.3. [**Recommandations générales**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/10_pitfalls_good_practises.ipynb#general-recommendations)
([*General recommendations*](https://scikit-learn.org/stable/common_pitfalls.html#general-recommendations))

# <a id='common-pitfalls-and-recommended-practices'></a> 10.1. Prétraitement incohérent

scikit-learn fournit une bibliothèque de [**transformations d'ensembles de données** (6)](https://scikit-learn.org/stable/data_transforms.html#data-transforms), qui peuvent nettoyer (voir [**Prétraitement des données** (6.3)](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing)), réduire (voir [**Réduction de dimensionnalité non supervisée** (6.5)](https://scikit-learn.org/stable/modules/unsupervised_reduction.html#data-reduction)), étendre (voir [**Approximation du noyau** (6.7)](https://scikit-learn.org/stable/modules/kernel_approximation.html#kernel-approximation)) ou générer (voir [**Extraction de caractéristiques** (6.2)](https://scikit-learn.org/stable/modules/feature_extraction.html#feature-extraction)) des représentations de caractéristiques. Si ces transformations de données sont utilisées lors de l'entraînement' d'un modèle, elles doivent également être utilisées sur les ensembles de données suivants, qu'il s'agisse de données de test ou de données dans un système de production. Sinon, l'espace des caractéristiques changera et le modèle ne pourra pas fonctionner efficacement.

Pour l'exemple suivant, créons un jeu de données synthétique avec une seule caractéristique :

In [2]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

random_state = 42
X, y = make_regression(random_state=random_state, n_features=1, noise=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=random_state)

**A ne pas faire**

L'ensemble de données d'entraînement est mis à l'échelle, mais pas l'ensemble de données de test, de sorte que les performances du modèle sur l'ensemble de données de test sont moins bonnes que prévu :

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
model = LinearRegression().fit(X_train_transformed, y_train)
mean_squared_error(y_test, model.predict(X_test))

62.80867119249539


**Correct**

Au lieu de passer le `X_test` non transformé à `predict`, nous devrions transformer les données de test, de la même manière que nous avons transformé les données d'apprentissage :

In [4]:
X_test_transformed = scaler.transform(X_test)
mean_squared_error(y_test, model.predict(X_test_transformed))

0.902797546636954

Alternativement, nous recommandons d'utiliser un [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline), qui facilite l'enchaînement des transformations avec les estimateurs, et réduit la possibilité d'oublier une transformation :

In [5]:
from sklearn.pipeline import make_pipeline #, Pipeline

model = make_pipeline(StandardScaler(), LinearRegression())
model.fit(X_train, y_train)
# Pipeline(steps=[('standardscaler', StandardScaler()),
#               ('linearregression', LinearRegression())])
mean_squared_error(y_test, model.predict(X_test))

0.902797546636954

Les pipelines permettent également d'éviter un autre écueil courant : la fuite des données de test dans les données d'apprentissage.

<a id='data-leakage'></a>

# 10.2. Fuite de données

Une fuite de données se produit lorsque des informations qui ne seraient pas disponibles au moment de la prédiction sont utilisées lors de la construction du modèle. Cela se traduit par des estimations de performances trop optimistes, par exemple à partir de la [3.1. validation croisée](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation), et donc des performances plus faibles lorsque le modèle est utilisé sur des données réellement nouvelles, par exemple pendant la production.

Une cause courante est de ne pas séparer les sous-ensembles de données de test et d'entraînement. Les données de test ne doivent jamais être utilisées pour faire des choix sur le modèle. **La règle générale est de ne jamais appeler** `fit` **sur les données de test**. Bien que cela puisse sembler évident, il est facile de passer à côté dans certains cas, par exemple lors de l'application de certaines étapes de prétraitement.

Bien que les sous-ensembles de données d'apprentissage et de test doivent recevoir la même transformation de prétraitement (comme décrit dans la section précédente), il est important que ces transformations ne soient apprises qu'à partir des données d'apprentissage. Par exemple, si vous avez une étape de normalisation dans laquelle vous divisez par la valeur moyenne, la moyenne doit être la moyenne du sous-ensemble d'nentraînement, et non la moyenne de toutes les données. Si le sous-ensemble de test est inclus dans le calcul de la moyenne, les informations du sous-ensemble de test influencent le modèle.

Un exemple de fuite de données lors du prétraitement est détaillé ci-dessous.

<a id='data-leakage-during-pre-processing'></a>

## 10.2.1. Fuite de données lors du prétraitement

**NB** - Nous choisissons ici d'illustrer la fuite de données avec une étape de sélection des caractéristiques. Ce risque de fuite est cependant pertinent avec presque toutes les transformations dans scikit-learn, y compris (mais sans s'y limiter) [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler), [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer) et [`PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA).

Un certain nombre de [1.13. fonctions de sélection de caractéristiques](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection) sont disponibles dans scikit-learn. Elles peuvent aider à supprimer les caractéristiques non pertinentes, redondantes et bruyantes, ainsi qu'à améliorer le temps de construction et les performances de votre modèle. Comme pour tout autre type de prétraitement, la sélection des caractéristiques ne doit utiliser que les données d'apprentissage. L'inclusion des données de test dans la sélection des caractéristiques biaisera votre modèle de manière optimiste.

Pour le démontrer, nous allons créer ce problème de classification binaire avec 10 000 caractéristiques générées aléatoirement :

In [6]:
import numpy as np
n_samples, n_features, n_classes = 200, 10000, 2
rng = np.random.RandomState(42)
X = rng.standard_normal((n_samples, n_features))
y = rng.choice(n_classes, n_samples)

**A ne pas faire**

L'utilisation de toutes les données pour effectuer la sélection des caractéristiques donne un score de précision bien supérieur au hasard, même si nos cibles sont complètement aléatoires. Ce caractère aléatoire signifie que nos `X` et `y` sont indépendants et nous nous attendons donc à ce que la précision soit d'environ 0,5. Cependant, puisque l'étape de sélection des caractéristiques "voit" les données de test, le modèle a un avantage injuste. Dans l'exemple incorrect ci-dessous, nous utilisons d'abord toutes les données pour la sélection des caractéristiques, puis nous divisons les données en sous-ensembles d'apprentissage et de test pour l'ajustement du modèle. Le résultat est un score de précision beaucoup plus élevé que prévu :

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Incorrect preprocessing: the entire data is transformed
X_selected = SelectKBest(k=25).fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, random_state=42)
gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train, y_train)
# GradientBoostingClassifier(random_state=1)

y_pred = gbc.predict(X_test)
accuracy_score(y_test, y_pred)

0.76

**Correct**

Pour éviter les fuites de données, il est recommandé de diviser d'abord vos données en sous-ensembles d'entraînement et de test. La sélection dde caractéristiques peut ensuite être entraînée en utilisant uniquement l'ensemble de données d'apprentissage. Notez que chaque fois que nous utilisons `fit` ou `fit_transform`, nous n'utilisons que le jeu de données d'apprentissage. Le score est à présent ce à quoi nous nous attendions pour les données, proche du hasard :

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)
select = SelectKBest(k=25)
X_train_selected = select.fit_transform(X_train, y_train)

gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train_selected, y_train)
# GradientBoostingClassifier(random_state=1)

X_test_selected = select.transform(X_test)
y_pred = gbc.predict(X_test_selected)
accuracy_score(y_test, y_pred)

0.46

Là encore, nous vous recommandons d'utiliser un [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) pour enchaîner la sélection de caractéristiques et les estimateurs de modèle. Le pipeline garantit que seules les données d'entraînement sont utilisées lors de l'ajustement et que les données de test sont utilisées uniquement pour calculer le score de précision :

In [9]:
from sklearn.pipeline import make_pipeline
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)
pipeline = make_pipeline(SelectKBest(k=25),
    GradientBoostingClassifier(random_state=1))
pipeline.fit(X_train, y_train)
# Pipeline(steps=[('selectkbest', SelectKBest(k=25)),
#                ('gradientboostingclassifier',
#                GradientBoostingClassifier(random_state=1))])

y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.46

Le pipeline peut également être introduit dans une fonction de validation croisée telle que [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score). Là encore, le pipeline garantit que le sous-ensemble de données et la méthode d'estimation corrects sont utilisés lors de l'ajustement et de la prédiction :

In [10]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, X, y)
print(f"Mean accuracy: {scores.mean():.2f}+/-{scores.std():.2f}")

Mean accuracy: 0.45+/-0.07


<a id='how-to-avoid-data-leakage'></a>

## 10.2.2. Comment éviter les fuites de données

Voici quelques conseils pour éviter les fuites de données :

* Commencez toujours par diviser les données en sous-ensembles d'entraînement et de test, en particulier avant toute étape de prétraitement.

* N'incluez jamais de données de test lorsque vous utilisez les méthodes `fit` et `fit_transform`. L'utilisation de toutes les données, par exemple `fit(X)`, peut entraîner des scores trop optimistes.

* À l'inverse, la méthode de transformation doit être utilisée à la fois sur les sous-ensembles d'entraînement et de test, car le même prétraitement doit être appliqué à toutes les données. Ceci peut être réalisé en utilisant `fit_transform` sur le sous-ensemble d'entraînement et `transform` sur le sous-ensemble de test.

* Le [6.1.1. pipeline](https://scikit-learn.org/stable/modules/compose.html#pipeline) scikit-learn est un excellent moyen d'éviter les fuites de données car il garantit que la méthode appropriée est exécutée sur le sous-ensemble de données approprié. Le pipeline est idéal pour une utilisation dans les fonctions de validation croisée et de réglage des hyperparamètres.

<a id='controlling-randomness'></a>

# 10.3. Contrôler l'aléatoire | Controlling randomness

Certains objets scikit-learn sont intrinsèquement aléatoires. Il s'agit généralement d'estimateurs (par exemple, [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)) et de séparateurs de validation croisée (par exemple, [`KFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold)). Le caractère aléatoire de ces objets est contrôlé via leur paramètre `random_state`, comme décrit dans le glossaire. Cette section développe l'entrée du glossaire et décrit les bonnes pratiques et les pièges courants par rapport à à ce paramètre subtil.

**NB** Résumé de la recommandation

Pour une robustesse optimale des résultats de la validation croisée (CV), transmettez des instances RandomState lors de la création d'estimateurs, ou laissez `random_state` sur None. Passer des entiers aux séparateurs de CV est généralement l'option la plus sûre et est préférable ; le passage d'instances RandomState à des séparateurs peut parfois être utile pour atteindre des cas d'utilisation très spécifiques. Pour les estimateurs et les séparateurs, la transmission d'un entier par rapport à la transmission d'une instance (ou `None`) entraîne des différences subtiles mais significatives, en particulier pour les procédures CV. Il est important de comprendre ces différences lors de la communication des résultats.

Pour des résultats reproductibles d'une exécution à l'autre, supprimez toute utilisation de `random_state=None`.

<a id='using-none-or-randomstate-instances-and-repeated-calls-to-fit-and-split'></a>

## 10.3.1. Utilisation d'instances `None` ou `RandomState`, et appels répétés à `fit` et `split`

Le paramètre `random_state` détermine si plusieurs appels à [`fit`](https://scikit-learn.org/stable/glossary.html#term-fit) (pour les estimateurs) ou à [`split`](https://scikit-learn.org/stable/glossary.html#term-split) (pour les séparateurs de CV) produiront les mêmes résultats, selon ces règles :

* Si un entier est passé, appeler `fit` ou `split` plusieurs fois donne toujours les mêmes résultats.
* Si `None` ou une instance `RandomState` est passée : `fit` et `split` donneront des résultats différents à chaque appel, et la succession d'appels explore toutes les sources d'entropie. `None` est la valeur par défaut pour tous les paramètres `random_state`.

Nous illustrons ici ces règles pour les estimateurs et les séparateurs de CV.

**NB** Puisque passer `random_state=None` équivaut à passer l'instance globale `RandomState` de `numpy` (`random_state=np.random.mtrand._rand`), nous ne mentionnerons pas explicitement `None` ici. Tout ce qui s'applique aux instances s'applique également à l'utilisation de `None`.

### Estimateurs

Passer des instances signifie qu'appeler `fit` plusieurs fois ne donnera pas les mêmes résultats, même si l'estimateur est ajusté sur les mêmes données et avec les mêmes hyper-paramètres :

In [1]:
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import make_classification
import numpy as np

rng = np.random.RandomState(0)
X, y = make_classification(n_features=5, random_state=rng)
sgd = SGDClassifier(random_state=rng)

sgd.fit(X, y).coef_
# array([[ 8.85418642,  4.79084103, -3.13077794,  8.11915045, -0.56479934]])

sgd.fit(X, y).coef_
# array([[ 6.70814003,  5.25291366, -7.55212743,  5.18197458,  1.37845099]])

array([[ 6.70814003,  5.25291366, -7.55212743,  5.18197458,  1.37845099]])

Nous pouvons voir à partir de l'extrait ci-dessus que l'appel répété à `sgd.fit` a produit différents modèles, même si les données étaient les mêmes. En effet, le générateur de nombres aléatoires (RNG) de l'estimateur est consommé (c'est-à-dire muté) lorsque `fit` est appelé, et ce RNG muté sera utilisé dans les appels ultérieurs à `fit`. De plus, l'objet `rng` est partagé par tous les objets qui l'utilisent et, par conséquent, ces objets deviennent quelque peu interdépendants. Par exemple, deux estimateurs qui partagent la même instance `RandomState` s'influenceront mutuellement, comme nous le verrons plus tard lorsque nous discuterons du clonage. Ce point est important à garder à l'esprit lors du débogage.

Si nous avions passé un entier au paramètre `random_state` du [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier), nous aurions obtenu les mêmes modèles, et donc les mêmes scores à chaque fois. Lorsque nous passons un entier, le même RNG est utilisé dans tous les appels à `fit`. Ce qui se passe en interne, c'est que même si le RNG est consommé lorsque `fit` est appelé, il est toujours réinitialisé à son état d'origine au début de `fit`.

### Séparateurs de CV

Les séparateurs de CV aléatoires ont un comportement similaire lorsqu'une instance `RandomState` est transmise ; appeler `split` plusieurs fois donne différentes divisions de données :

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

X = y = np.arange(10)
rng = np.random.RandomState(0)
cv = KFold(n_splits=2, shuffle=True, random_state=rng)

for train, test in cv.split(X, y):
        print(train, test)
# [0 3 5 6 7] [1 2 4 8 9]
# [1 2 4 8 9] [0 3 5 6 7]

for train, test in cv.split(X, y):
        print(train, test)
# [0 4 6 7 8] [1 2 3 5 9]
# [1 2 3 5 9] [0 4 6 7 8]

Nous pouvons voir que les divisions sont différentes à partir de la deuxième fois que `split` est appelée. Cela peut conduire à des résultats inattendus si vous comparez les performances de plusieurs estimateurs en appelant plusieurs fois `split`, comme nous le verrons dans la section suivante.

<a id='common-pitfalls-and-subtleties'></a>

## 10.3.2. Pièges et subtilités courants

Bien que les règles qui régissent le paramètre `random_state` soient apparemment simples, elles ont cependant des implications subtiles. Dans certains cas, cela peut même conduire à des conclusions erronées.

### Estimateurs

#### Différents types ``random_state`` conduisent à différentes procédures de validation croisée

Selon le type du paramètre `random_state`, les estimateurs se comporteront différemment, en particulier dans les procédures de validation croisée. Considérez l'extrait suivant :

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
import numpy as np

X, y = make_classification(random_state=0)

rf_123 = RandomForestClassifier(random_state=123)
cross_val_score(rf_123, X, y)
# array([0.85, 0.95, 0.95, 0.9 , 0.9 ])

rf_inst = RandomForestClassifier(random_state=np.random.RandomState(0))
cross_val_score(rf_inst, X, y)
# array([0.9 , 0.95, 0.95, 0.9 , 0.9 ])

array([0.9 , 0.95, 0.95, 0.9 , 0.9 ])

Nous voyons que les scores validés croisés de `rf_123` et `rf_inst` sont différents, comme on pouvait s'y attendre puisque nous n'avons pas passé le même paramètre `random_state`. Cependant, la différence entre ces scores est plus subtile qu'il n'y paraît, et **les procédures de validation croisée qui ont été effectuées par [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) diffèrent considérablement dans chaque cas** :

* Puisque `rf_123` a reçu un entier, chaque appel à fit utilise le même RNG : cela signifie que toutes les caractéristiques aléatoires de l'estimateur de forêt aléatoire seront les mêmes pour chacun des 5 plis de la procédure CV. En particulier, le sous-ensemble (choisi au hasard) de caractéristiques de l'estimateur sera le même dans tous les plis.

* Étant donné que `rf_inst` a reçu une instance `RandomState`, chaque appel à `fit` démarre à partir d'un RNG différent. En conséquence, le sous-ensemble aléatoire de caractéristiques sera différent pour chaque pli.

Bien qu'avoir un RNG d'estimateur constant à travers les plis ne soit pas intrinsèquement faux, nous voulons généralement des résultats de CV qui sont robustes en ce qui concerne le caractère aléatoire de l'estimateur. Par conséquent, passer une instance au lieu d'un entier peut être préférable, car cela permettra à l'estimateur RNG de varier pour chaque pli.

#### Clonage

Un autre effet secondaire subtil du passage d'instances `RandomState` est le fonctionnement de `clone` :

In [3]:
from sklearn import clone
from sklearn.ensemble import RandomForestClassifier
import numpy as np

rng = np.random.RandomState(0)
a = RandomForestClassifier(random_state=rng)
b = clone(a)

Puisqu'une instance `RandomState` a été passée à `a`, `a` et `b` ne sont pas des clones au sens strict, mais plutôt des clones au sens statistique : `a` et `b` seront toujours des modèles différents, même en appelant `fit(X, y)` sur les mêmes données . De plus, `a` et `b` s'influenceront mutuellement puisqu'ils partagent le même RNG interne : appeler `a.fit` consommera le RNG de `b`, et appeler `b.fit` consommera le RNG de `a`, puisqu'ils sont identiques. Ce petit peu est vrai pour tous les estimateurs qui partagent un paramètre `random_state` ; elle n'est pas spécifique aux clones.

Si un entier était passé, `a` et `b` seraient des clones exacts et ils ne s'influenceraient pas l'un l'autre.

**Avertissement** - Même si `clone` est rarement utilisé dans le code utilisateur, il est appelé de manière omniprésente dans la base de code scikit-learn : en particulier, la plupart des méta-estimateurs qui acceptent des estimateurs non ajustés appellent `clone` en interne ([`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV), [`StackingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier), [`CalibratedClassifierCV`](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html#sklearn.calibration.CalibratedClassifierCV), etc.).

### Séparateurs de CV

Lorsqu'une instance `RandomState` est transmise, les séparateurs de CV génèrent des divisions différentes à chaque fois que `split` est appelée. Lorsque l'on compare différents estimateurs, cela peut conduire à surestimer la variance de la différence de performance entre les estimateurs :

In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import numpy as np

rng = np.random.RandomState(0)
X, y = make_classification(random_state=rng)
cv = KFold(shuffle=True, random_state=rng)
lda = LinearDiscriminantAnalysis()
nb = GaussianNB()

for est in (lda, nb):
    print(cross_val_score(est, X, y, cv=cv))
# [0.8  0.75 0.75 0.7  0.85]
# [0.85 0.95 0.95 0.85 0.95]

[0.8  0.75 0.75 0.7  0.85]
[0.85 0.95 0.95 0.85 0.95]


Comparer directement les performances de l'estimateur [`LinearDiscriminantAnalysis`](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html#sklearn.discriminant_analysis.LinearDiscriminantAnalysis) vs l'estimateur [`GaussianNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB) sur **chaque pli** serait une erreur : **les splits sur lesquels les estimateurs sont évalués sont différents**. En effet, [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) appellera en interne `cv.split` sur la même instance de [`KFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold), mais les splits seront différents à chaque fois. Cela est également vrai pour tout outil qui effectue une sélection de modèle via une validation croisée, par ex. [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) et [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV) : les scores ne sont pas comparables entre les différents appels à `search.fit`, car `cv.split` aurait été appelé plusieurs fois. Dans un seul appel à `search.fit`, cependant, la comparaison de pli à pli est possible puisque l'estimateur de recherche n'appelle `cv.split` qu'une seule fois.

Pour obtenir des résultats pli à pli comparables dans tous les scénarios, il convient de transmettre un entier au séparateur de CV : `cv = KFold(shuffle=True, random_state=0)`.

**NB** - Bien que la comparaison pli à pli ne soit pas recommandée avec les instances `RandomState`, on peut cependant s'attendre à ce que les scores moyens permettent de conclure si un estimateur est meilleur qu'un autre, tant que suffisamment de plis et de données sont utilisés.

**NB** - Ce qui compte dans cet exemple, c'est ce qui a été passé à [`KFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold). Que nous transmettions une instance `RandomState` ou un entier à [`make_classification`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html#sklearn.datasets.make_classification) n'est pas pertinent pour notre objectif d'illustration. De plus, ni [`LinearDiscriminantAnalysis`](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html#sklearn.discriminant_analysis.LinearDiscriminantAnalysis) ni [`GaussianNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB) ne sont des estimateurs randomisés.

<a id='general-recommendations'></a>

## 10.3.3. Recommandations générales

<a id='getting-reproducible-results-across-multiple-executions'></a>

### Obtenir des résultats reproductibles sur plusieurs exécutions

Afin d'obtenir des résultats reproductibles (c'est-à-dire constants) sur plusieurs *exécutions de programme*, nous devons supprimer toutes les utilisations de `random_state=None`, qui est la valeur par défaut. La méthode recommandée consiste à déclarer une variable `rng` en haut du programme et à la transmettre à tout objet acceptant un paramètre `random_state` :

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np

rng = np.random.RandomState(0)
X, y = make_classification(random_state=rng)
rf = RandomForestClassifier(random_state=rng)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=rng)
rf.fit(X_train, y_train).score(X_test, y_test)
# 0.84

0.84

Nous sommes désormais assurés que le résultat de ce script sera toujours de 0,84, quel que soit le nombre de fois que nous l'exécuterons. La modification de la variable globale `rng` à une valeur différente devrait affecter les résultats, comme prévu.

Il est également possible de déclarer la variable `rng` comme un entier. Cela peut cependant conduire à des résultats de validation croisée moins robustes, comme nous le verrons dans la section suivante.

**NB** - Nous ne recommandons pas de définir la graine `numpy` globale en appelant `np.random.seed(0)`. Voir [ici](https://stackoverflow.com/questions/5836335/consistently-create-same-random-numpy-array/5837352#comment6712034_5837352) pour une discussion.

<a id='robustness-of-cross-validation-results'></a>

### Robustesse des résultats de la validation croisée

Lorsque nous évaluons les performances d'un estimateur randomisé par validation croisée, nous voulons nous assurer que l'estimateur peut produire des prédictions précises pour de nouvelles données, mais nous voulons également nous assurer que l'estimateur est robuste en ce qui concerne son initialisation aléatoire. Par exemple, nous aimerions que l'initialisation des poids aléatoires d'un `SGDCLassifier` soit toujours bonne dans tous les plis : sinon, lorsque nous entraînons cet estimateur sur de nouvelles données, nous risquons d'avoir de la malchance et l'initialisation aléatoire peut entraîner de mauvaises performances. De même, nous voulons qu'une forêt aléatoire soit robuste par rapport à l'ensemble de caractéristiques sélectionnées au hasard que chaque arbre utilisera.

Pour ces raisons, il est préférable d'évaluer les performances de la validation croisée en laissant l'estimateur utiliser un RNG différent sur chaque pli. Cela se fait en passant une instance `RandomState` (ou `None`) à l'initialisation de l'estimateur.

Lorsque nous passons un entier, l'estimateur utilisera le même RNG sur chaque pli : si l'estimateur fonctionne bien (ou mal), tel qu'évalué par CV, c'est peut-être simplement parce que nous avons eu de la chance (ou de la malchance) avec cette graine spécifique. Le passage d'instances conduit à des résultats de CV plus robustes et rend la comparaison entre différents algorithmes plus juste. **Cela aide également à limiter la tentation de traiter le RNG de l'estimateur comme un hyper-paramètre qui peut être réglé**.

Que nous transmettions des instances `RandomState` ou des entiers aux séparateurs de CV n'a aucun impact sur la robustesse, tant que split n'est appelé qu'une seule fois. Lorsque split est appelé plusieurs fois, la comparaison pli à pli n'est plus possible. Par conséquent, le passage d'un entier aux séparateurs de CV est généralement plus sûr et couvre la plupart des cas d'utilisation.